In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

PATH = "D:\\FAST.Ai\\Machine Learning for coders\\Data\\"

In [3]:
df_raw = pd.read_csv(f'{PATH}bulldozers.txt', low_memory=False, 
                     parse_dates=["saledate"])

In [24]:
df_raw.shape

(51956, 65)

In [5]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

display_all(df_raw.tail().T)

,51951,51952,51953,51954,51955
SalesID,1294835,1294836,1294838,1294840,1294841
SalePrice,9500,12250,13750,8500,11000
MachineID,1542283,1260127,1330996,1524585,1212936
ModelID,3150,3150,3150,945,945
datasource,132,132,132,132,132
auctioneerID,1,99,10,1,1
YearMade,1991,1992,1992,1992,1000
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2007-06-28 00:00:00,2006-10-26 00:00:00,1998-06-25 00:00:00,2005-09-29 00:00:00,2004-05-21 00:00:00


In [6]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [7]:
add_datepart(df_raw,"saledate") # adding year , month , ...   instead of just a date 

In [8]:
train_cats(df_raw) 

In [9]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium' , 'Low'] , inplace = True , ordered = True)
# changing order of assigning nubers to ['High', 'Medium' , 'Low'] 

In [15]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

In [17]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [18]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 12000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

In [19]:
set_rf_samples(50000)

In [20]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(x_train, y_train)
print_score(m)

[0.11136420048635667, 0.20191316927312813, 0.976390901592768, 0.6417813485948838, 0.9109934301182017]


In [21]:
from scipy.cluster import hierarchy as hc

In [23]:
corr = np.round(scipy.stats.spearmanr(df_trn).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
fig = plt.figure(figsize=(16,10))
dendrogram = hc.dendrogram(z, labels=df_keep.columns, orientation='left', leaf_font_size=16)
plt.show()

C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\numpy\lib\function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\numpy\lib\function_base.py:2401: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Lenovo\Anaconda3\envs\fastai\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


ValueError: Distance matrix 'X' must be symmetric.